In [1]:
# Entities
xapps = ["x1", "x2", "x3", "x4"]
parameters = ["p1", "p2", "p3"]
kpis = ["k1", "k2", "k3"]

# Global parameter domains
param_domains = {
    "p1": (0, 40),
    "p2": (0, 100),
    "p3": (-10, 10)
}
# Global KPI ranges (system-defined)
kpi_domains = {
    "k1": (50, 200),   # Throughput (Mbps)
    "k2": (5, 20),     # Latency (ms)
    "k3": (0, 2)       # Packet loss (%)
}

In [2]:
xapps = ["x1", "x2", "x3", "x4"]
parameters = list(param_domains.keys())
kpis = list(kpi_domains.keys())


In [3]:
import random
import pandas as pd
import numpy as np

def random_range(lo, hi, min_width=0.2):
    a = random.uniform(lo, hi - min_width)
    b = random.uniform(a + min_width, hi)
    return round(a, 2), round(b, 2)


In [5]:
xapp_param_data = []

for x in xapps:
    controlled_params = random.sample(parameters, random.randint(1, 2))
    for p in controlled_params:
        lo, hi = param_domains[p]
        rmin, rmax = random_range(lo, hi)
        xapp_param_data.append([x, p, rmin, rmax])

xapp_param_df = pd.DataFrame(
    xapp_param_data,
    columns=["xapp", "parameter", "range_min", "range_max"]
)

print("\nXApp–Parameter control ranges")
print(xapp_param_df)



XApp–Parameter control ranges
  xapp parameter  range_min  range_max
0   x1        p3       2.56       8.39
1   x1        p1      29.11      39.93
2   x2        p2      27.80      45.47
3   x2        p1      10.93      21.37
4   x3        p3      -2.51       1.28
5   x4        p3      -6.18       3.04
6   x4        p1      20.71      27.67


In [6]:
param_kpi_data = []

for p in parameters:
    affected_kpis = random.sample(kpis, random.randint(1, 2))
    for k in affected_kpis:
        plo, phi = param_domains[p]
        klo, khi = kpi_domains[k]

        # parameter sub-range where it impacts KPI
        pmin, pmax = random_range(plo, phi)

        param_kpi_data.append([p, pmin, pmax, k, klo, khi])

param_kpi_df = pd.DataFrame(
    param_kpi_data,
    columns=[
        "parameter",
        "param_range_min", "param_range_max",
        "kpi",
        "kpi_global_min", "kpi_global_max"
    ]
)

print("\nParameter–KPI influence ranges")
print(param_kpi_df)



Parameter–KPI influence ranges
  parameter  param_range_min  param_range_max kpi  kpi_global_min  \
0        p1            29.42            39.25  k1              50   
1        p1            29.50            37.69  k2               5   
2        p2            23.84            83.03  k2               5   
3        p2             2.16            55.97  k1              50   
4        p3            -8.39             0.94  k2               5   

   kpi_global_max  
0             200  
1              20  
2              20  
3             200  
4              20  


In [7]:
I_xp = pd.DataFrame(0, index=xapps, columns=parameters)

for _, row in xapp_param_df.iterrows():
    I_xp.loc[row["xapp"], row["parameter"]] = 1

print("\nI_xp (xApp–Parameter incidence)")
print(I_xp)



I_xp (xApp–Parameter incidence)
    p1  p2  p3
x1   1   0   1
x2   1   1   0
x3   0   0   1
x4   1   0   1


In [8]:
I_pk = pd.DataFrame(0, index=parameters, columns=kpis)

for _, row in param_kpi_df.iterrows():
    I_pk.loc[row["parameter"], row["kpi"]] = 1

print("\nI_pk (Parameter–KPI incidence)")
print(I_pk)



I_pk (Parameter–KPI incidence)
    k1  k2  k3
p1   1   1   0
p2   1   1   0
p3   0   1   0
